In [4]:
!wget https://github.com/Ankit152/IMDB-sentiment-analysis/raw/master/IMDB-Dataset.csv

--2022-06-11 16:48:32--  https://github.com/Ankit152/IMDB-sentiment-analysis/raw/master/IMDB-Dataset.csv
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv [following]
--2022-06-11 16:48:32--  https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [text/plain]
Saving to: ‘IMDB-Dataset.csv’

IMDB-Dataset.csv    100%[===================>]  63.14M   197MB/s    in 0.3s    

2022-06-11 16:48:36 (197 MB/s) - ‘IMDB-Dataset.csv’ saved [66212309/66212309

In [5]:
import pandas as pd
imdb_df = pd.read_csv("IMDB-Dataset.csv")
reviews = imdb_df.review.to_string(index=None)
with open("corpus.txt", "w") as f:
      f.writelines(reviews)

In [4]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 34.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 56.4 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 764 kB 60.6 MB/s eta 0:00:01


In [1]:
from tokenizers import BertWordPieceTokenizer
bert_wordpiece_tokenizer = BertWordPieceTokenizer()
bert_wordpiece_tokenizer.train("corpus.txt")

In [6]:
bert_wordpiece_tokenizer.get_vocab()

{'noise': 8923,
 '##bell': 5915,
 'cheers': 15459,
 'extras': 17485,
 'hopeless': 12770,
 ',': 16,
 'leg': 1800,
 'trendy': 17337,
 '##anist': 14460,
 'electric': 13321,
 'spectac': 3530,
 '##ional': 1921,
 'nat': 2084,
 'hobgob': 9077,
 '##antic': 8001,
 'tights': 10959,
 '##prise': 5284,
 'duch': 5643,
 'mccarthy': 13713,
 'georget': 17652,
 'swed': 2764,
 '##aaa': 12923,
 'lucky': 2252,
 '##gerald': 12707,
 'chun': 11882,
 '##endium': 15198,
 'bearing': 16782,
 'updating': 9508,
 'hair': 3913,
 'min': 573,
 'mockument': 6510,
 'satur': 2057,
 'haneke': 9694,
 'frank': 1537,
 'ludic': 5420,
 'vers': 750,
 'ev': 866,
 '##off': 3044,
 '##enth': 11807,
 '##lland': 11805,
 '##imi': 14703,
 'stay': 2271,
 'diner': 16656,
 'oklahoma': 17937,
 'instantly': 10698,
 'ketchup': 18090,
 'videostore': 15527,
 '##pa': 5810,
 'chave': 14758,
 '##self': 865,
 'corrupt': 9186,
 'grind': 10333,
 'peoples': 10297,
 'unique': 2011,
 'obel': 12414,
 'jung': 4652,
 'had': 315,
 'kristoff': 9472,
 'distin

In [9]:
bert_wordpiece_tokenizer.save_model("tokenizer")

['tokenizer/vocab.txt']

In [7]:
tokenizer = BertWordPieceTokenizer.from_file("tokenizer/vocab.txt")

In [8]:
tokenized_sentence = \
tokenizer.encode("Oh it works just fine")
tokenized_sentence.tokens

['[CLS]', 'oh', 'it', 'works', 'just', 'fine', '[SEP]']

In [9]:
tokenized_sentence = \
tokenizer.encode("ohoh i thougt it might be workingg well")
tokenized_sentence.tokens

['[CLS]',
 'oh',
 '##o',
 '##h',
 'i',
 'thoug',
 '##t',
 'it',
 'might',
 'be',
 'working',
 '##g',
 'well',
 '[SEP]']

In [19]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("tokenizer")

Didn't find file tokenizer/tokenizer.json. We won't load it.
Didn't find file tokenizer/added_tokens.json. We won't load it.
Didn't find file tokenizer/special_tokens_map.json. We won't load it.
Didn't find file tokenizer/tokenizer_config.json. We won't load it.
loading file tokenizer/vocab.txt
loading file None
loading file None
loading file None
loading file None


In [11]:
from transformers import LineByLineTextDataset
dataset = \
LineByLineTextDataset(tokenizer=tokenizer,
                      file_path="corpus.txt", 
                      block_size=128)

/opt/conda/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
                      tokenizer=tokenizer, 
                      mlm=True, 
                      mlm_probability=0.15)

In [13]:
from transformers import TrainingArguments
training_args = TrainingArguments(
                      output_dir="BERT",
                      overwrite_output_dir=True,
                      num_train_epochs=1,
                      per_device_train_batch_size=128)

In [14]:
from transformers import BertConfig, BertForMaskedLM
bert = BertForMaskedLM(BertConfig())

In [15]:
from transformers import Trainer
trainer = Trainer(model=bert, 
                      args=training_args,
                      data_collator=data_collator,
                      train_dataset=dataset)

In [16]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 50022
  Num Epochs = 1
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 391


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=391, training_loss=5.366514396179667, metrics={'train_runtime': 263.1469, 'train_samples_per_second': 190.092, 'train_steps_per_second': 1.486, 'total_flos': 635727015135000.0, 'train_loss': 5.366514396179667, 'epoch': 1.0})

In [17]:
trainer.save_model("MyBERT")

Saving model checkpoint to MyBERT
Configuration saved in MyBERT/config.json
Model weights saved in MyBERT/pytorch_model.bin


In [20]:
import torch
torch.cuda.is_available()

True

In [2]:
from transformers import\
TFBertModel, BertTokenizerFast

2022-06-11 17:30:50.048247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 17:30:50.464076: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 17:30:50.464941: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 17:30:50.466286: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
bert = TFBertModel.from_pretrained("bert-base-uncased")
tokenizer2 = BertTokenizerFast.from_pretrained(
"bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [4]:
bert.layers

In [16]:
tokenized_text = tokenizer2.batch_encode_plus(
                   ["hello how is it going with you",
                   "lets test it"], 
                    return_tensors="tf", 
                    max_length=128, 
                    truncation=True, 
                    pad_to_max_length=True)

In [17]:
bert(tokenized_text)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[ 0.10047169,  0.0677025 , -0.08335952, ..., -0.4933047 ,
          0.11653915,  0.22664754],
        [ 0.3236246 ,  0.37071785,  0.61468565, ..., -0.62726766,
          0.37908283,  0.07053157],
        [ 0.19953474, -0.87550944, -0.06478716, ..., -0.0128083 ,
          0.30765152, -0.02073251],
        ...,
        [-0.20290422,  0.15799463,  0.60413766, ..., -0.17316641,
         -0.02515502,  0.19266947],
        [-0.1967066 ,  0.03543479,  0.65431905, ..., -0.07587673,
         -0.06067057,  0.03544359],
        [-0.24859604,  0.0245423 ,  0.6876988 , ..., -0.08229918,
         -0.05705748,  0.05443959]],

       [[ 0.02945649,  0.2308684 ,  0.292652  , ..., -0.13042173,
          0.18965931,  0.46842816],
        [ 1.7052323 ,  0.69135946,  0.7315114 , ...,  0.2893024 ,
          0.53675807, -0.15455289],
        [ 0.10459755,  0.09636812,  0.06996638, .

In [18]:
from tensorflow import keras
import tensorflow as tf
max_length = 128
tokens = keras.layers.Input(shape=(max_length,),
                           dtype=tf.dtypes.int32)
masks = keras.layers.Input(shape=(max_length,),
                          dtype=tf.dtypes.int32)
embedding_layer = bert.layers[0]([tokens,masks])[0][:,0,:]
dense = tf.keras.layers.Dense(units=2, 
        activation="softmax")(embedding_layer)
model = keras.Model([tokens,masks],dense)

In [19]:
tokenized = tokenizer2.batch_encode_plus(
["hello how is it going with you",
"hello how is it going with you"], 
return_tensors="tf", 
max_length= max_length, 
truncation=True, 
pad_to_max_length=True)

In [20]:
model([tokenized["input_ids"],tokenized["attention_mask"]])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.49622813, 0.50377184],
       [0.49622813, 0.50377184]], dtype=float32)>

In [21]:
model.compile(optimizer="Adam",
loss="categorical_crossentropy", 
metrics=["accuracy"])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          multiple             109482240   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 768)          0           bert[2][0]                 

In [22]:
model.layers[2].trainable = False

In [23]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          multiple             109482240   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 768)          0           bert[2][0]                 

In [24]:
import pandas as pd
imdb_df = pd.read_csv("IMDB-Dataset.csv")
reviews = list(imdb_df.review)
tokenized_reviews = \
tokenizer2.batch_encode_plus(reviews, return_tensors="tf",
                           max_length= max_length,
                           truncation=True, 
                           pad_to_max_length=True)
import numpy as np
train_split = int(0.8 * len(tokenized_reviews["attention_mask"]))
train_tokens = tokenized_reviews["input_ids"][:train_split]
test_tokens = tokenized_reviews["input_ids"][train_split:]
train_masks = tokenized_reviews["attention_mask"][:train_split]
test_masks = tokenized_reviews["attention_mask"][train_split:]
sentiments = list(imdb_df.sentiment)
labels = np.array([[0,1] if sentiment == "positive" else [1,0] for sentiment in sentiments])
train_labels = labels[:train_split]
test_labels = labels[train_split:]

In [25]:
model.fit([train_tokens,train_masks],train_labels, 
             epochs=5)

Epoch 1/5
1250/1250 [==============================] - 387s 304ms/step - loss: 0.4880 - accuracy: 0.7669
Epoch 2/5
1250/1250 [==============================] - 387s 309ms/step - loss: 0.4566 - accuracy: 0.7862
Epoch 3/5
1250/1250 [==============================] - 385s 308ms/step - loss: 0.4497 - accuracy: 0.7902
Epoch 4/5
1250/1250 [==============================] - 386s 309ms/step - loss: 0.4503 - accuracy: 0.7903
Epoch 5/5
1250/1250 [==============================] - 386s 309ms/step - loss: 0.4504 - accuracy: 0.7921


In [26]:
model